In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación 

### Minería de datos: Curso 2020-2021 

* José Gabriel Ruiz Gomez
* Francisco Javier Vicente Martínez

**Base de datos Wisconsin**


# 1. Preliminares

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
sns.set()
from sklearn.impute import SimpleImputer

#IMPORTANTE AÑADIRLO EN NUESTRO LOCAL
# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

In [ ]:
#Fijación de la semilla 
seed = 27912

# 2. Acceso y almacenamiento de datos

El conjunto datos a utilizar es `Breast Cancer Wisconsin`. 
Contiene 546 muestras que se clasifican en dos tipos de tumores:
* `B = Benigno`
* `M = Maligno`

Para cada tumor se han realizado una serie de mediciones correspondientes a las variables predictoras del problema:
* `radius` : media de distancias entre el centro de los puntos al perímetro
* `texture` : desviación estandard de los valores de escala de grises
* `perimeter` : perímetro
* `area` : área
* `smoothness` : variación local en la longitud de los radios
* `compactness` : perímetro^2 / area - 1.0
* `concavity` : severidad de las porciones cóncavas del contorno
* `concave points` : numero de las porciones cóncavas del contorno
* `symetry` : simetría
* `fractal dimension` : aproximación de la línea de costa - 1

En las tablas se representa la media, la desviación típica y un valor "worst" que es la media de los tres mayores valores. Resultando 30 variables distribuidas de forma que el campo 3 es Media del Radio, el campo 13 desviación típica del Radio y campo 23 es "worst" del radio.

El objetivo sería clasificar una nueva instancia (cuya clasificación es desconocida) en función de sus variables.



Comenzamos cargando el conjunto de datos `Breast Cancer Wisconsin`:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Dividimos el conjunto de datos en dos subconjuntos, uno con variables predictoras (X) y otro con la variable objetivo (y).

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Además debemos separar nuestro conjunto de datos en dos:

* Una muestra de entrenamiento (típicamente, 70%)
* Una muestra de prueba (típicamente, 30%)

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                     stratify=y, 
                                                     random_state=seed,
                                                     train_size=train_size)


Para facilitar el análisis exploratorio de datos, volvemos a juntar las variables predictoras con la variable clase. Comenzamos con el conjunto de datos entrenamiento:

In [ ]:
data_train = utils.join_dataset(X_train, y_train)

Continuamos con el conjunto de datos de prueba:

In [ ]:
data_train

In [ ]:
data_test = utils.join_dataset(X_test, y_test)

# 3. Análisis exploratorio de datos

Antes de comenzar el preprocesamiento es interesante observar las propiedades del conjunto de datos, analizando sus variables y la interacción entre estas. No obstante, no podemos usar el formato tabular directamente puesto que para un humano es casi imposible extraer conclusiones a partir del análisis de valores numéricos. Por ello, nos apoyaremos en gráficos y estadísticos.

### Descripción del conjunto de datos

Antes de realizar cualquier operación es fundamental conocer nuestro problema. Hay dos dimensiones básicas que deben ser exploradas:

* Número de casos
* Número de variables
    * Tipo de las variables: Continuas (t.c.c. numéricas) o discretas (t.c.c. categóricas)

Para ello, consultaremos las estructuras de datos correspondientes.

In [ ]:
print(X_train.shape)
print(y_train.shape)

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 398 casos y 31 variables (30 variables predictoras y 1 variable clase)

Para conocer cuál es el tipo de las variables, recurrimos al método `info`:

In [ ]:
data_train.info(memory_usage=False)

Todas las variables son numéricas, excepto la variable objetivo. 

In [ ]:
y_train.cat.categories

La variable clase tiene dos estados *(M y B)*.

### Visualización de las variables

Una vez conocemos con más detalle el conjunto de datos de entrenamiento, lo que debemos hacer es representar y analizar las distribuciones de las variables. Para ello, utilizaremos métodos univariados, esto es, histogramas para las variables numéricas y diagramas de barras para las variables categóricas. En particular:

* Un histograma muestra la densidad de ejemplos para los distintos valores de una variable numérica.
* Un diagrama de barras representa la frecuencia de cada estado de una variable categórica.

In [ ]:
utils.plot_histogram(X_train)

In [ ]:
utils.plot_barplot(data)

Podemos ver que la clase objetivo no tiene el mismo número de variables para cada clasificación por lo tanto no es un problema que se encuentre balanceado. 

In [ ]:
data_train.describe(include="category")

In [ ]:
data_train.describe(include="number")

Para que las gráficas sean más representativas dividimos el conjunto de datos de entrenamiento en tres subconjuntos, que representarán cada uno los valores de las medias (means_data), los valores de el error standard (SE_data) y los valores "worst" (worst_data)

In [ ]:
means_data = data_train[["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean", "concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","diagnosis"]]
SE_data = data_train[["radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se", "concavity_se","concave points_se","symmetry_se","fractal_dimension_se","diagnosis"]]
worst_data = data_train[["radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst", "concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst","diagnosis"]]

Ahora representaremos en diferentes gráficas los datos para obtener más conocimiento

In [ ]:
sns.heatmap(means_data.corr(), annot=True)


In [ ]:
sns.heatmap(SE_data.corr(), annot=True)

In [ ]:
sns.heatmap(worst_data.corr(), annot=True)

In [ ]:
sp = utils.plot_pairplot(means_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

In [ ]:
sp = utils.plot_pairplot(SE_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

In [ ]:
sp = utils.plot_pairplot(worst_data, target="diagnosis")
sp.update_layout(width=1400, height=1400, hovermode='closest')
sp.show()

# 4. Preprocesamiento de datos

Dentro del preprocesamiento de datos, podemos destacar las siguientes tareas:

* Limpieza de datos (imputación de valores perdidos, suavizado del ruido, etc.)
* Integración de datos (a partir de múltiples fuentes)
* Transformación de datos (normalización, construcción, etc.)
* Reducción de datos (discretización de variables numéricas, selección de variables, selección de instancias, etc.)

### Valores perdidos

Para poder usar un pipeline y poder distinguir las variables con valores perdidos tenemos que "modificar" toda la base de datos, realmente no vamos a modificar los datos, simplemente vamos a indicar como nan los valores perdidos, de esta forma podemos aplicar el mismo pipeline al conjunto de entrenamiento, al conjunto de test y a los datos que vengan despues.

In [ ]:
X_train.sample(100, random_state=seed)

### Discretización

Como hemos visto, la discretización permite transformar variables numéricas en categóricas, siendo este paso beneficioso para algunos algoritmos de aprendizaje, pues permite que modelos lineales resuelvan problemas no lineales.

`scikit-learn` permite realizar tres tipos de discretización (`strategy`) mediante el transformador `KBinsDiscretizer`:

* `uniform`: Igual anchura.
* `quantile`: Igual frecuencia.
* `kmeans`: Discretización basada en k-medias.

Tras el análisis exploratorio de datos realizado previamente, parece lógico realizar una discretización en 3 intevalos de igual anchura:

In [ ]:
#imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
#imputer = SimpleImputer(missing_values=np.NaN, strategy='mean')
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')

In [ ]:
discretizer = KBinsDiscretizer(n_bins=2, strategy="kmeans")

# 5. Algoritmos de clasificación

### Algoritmo Zero-R

In [ ]:
zero_r_model = make_pipeline(imputer, DummyClassifier(strategy="most_frequent"))

### Algoritmo CART 

In [ ]:
tree_model = make_pipeline(imputer, DecisionTreeClassifier(random_state=seed))

In [ ]:
discretize_tree_model = make_pipeline(imputer, discretizer, DecisionTreeClassifier(random_state=seed))

# 6. Evaluación de modelos

### Zero-R

In [ ]:
utils.evaluate(zero_r_model,
              X_train, X_test,
              y_train, y_test)

### Arbol de clasificación

In [ ]:
utils.evaluate(tree_model,
              X_train, X_test,
              y_train, y_test)

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

De los tres algoritmos evaluados el que mayor precisión tiene es cuando hemos discretizado los valores. Sin embargo, en la aplicación real de este problema nos parece que es mejor la estrategia sin discretizar ya que nuestro objetivo sería minimizar el número de falsos negativos, es decir, que el algoritmo te clasifique como Benigno y realmente sea Maligno, para este caso sin discretizar tenemos mejores resultados. 